In [1]:
!pip install folium

import folium
import json
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Function to get earthquake data from USGS GeoJSON Feed
def get_earthquake_data():
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
    response = requests.get(url)
    data = response.json()
    return data


In [3]:
# Function to create the map
def create_map():
    # Get earthquake data
    data = get_earthquake_data()

    # Create map centered on United States
    m = folium.Map(location=[37.09, -95.71], zoom_start=4)

    # Add tile layer
    folium.TileLayer("cartodbpositron").add_to(m)

    # Iterate over earthquake features and add markers to the map
    for feature in data["features"]:
        properties = feature["properties"]
        coordinates = feature["geometry"]["coordinates"]
        magnitude = properties["mag"]
        depth = coordinates[2]

        # Define marker size based on magnitude
        marker_size = magnitude * 5

        # Define marker color based on depth
        if depth < 10:
            marker_color = "green"
        elif depth < 30:
            marker_color = "yellow"
        else:
            marker_color = "red"

        # Create marker with tooltip
        folium.CircleMarker(
            location=[coordinates[1], coordinates[0]],
            radius=marker_size,
            color=marker_color,
            fill=True,
            fill_opacity=0.7,
            fill_color=marker_color,
            tooltip=f"Magnitude: {magnitude}<br>Location: {properties['place']}<br>Depth: {depth} km"
        ).add_to(m)

    # Add legend
    legend_html = """
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px;">
            <h4>Legend</h4>
            <p><strong>Depth</strong></p>
            <p><span style="background-color:green;">&nbsp;&nbsp;&nbsp;</span> &lt; 10 km</p>
            <p><span style="background-color:yellow;">&nbsp;&nbsp;&nbsp;</span> 10 - 30 km</p>
            <p><span style="background-color:red;">&nbsp;&nbsp;&nbsp;</span> &gt; 30 km</p>
        </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

In [4]:
# Create the map
map = create_map()

In [5]:
# Display the map
map